In [1]:
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
!pip install -q transformers==4.36.0
!pip install -q torch torchvision
!pip install -q pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.0 which is incompatible.


In [11]:
import os
os.environ["OPENAI_API_KEY"] = "your-api-key-here"
api_key = os.getenv("OPENAI_API_KEY")

In [12]:
import openai

# Set the OpenAI API key directly
openai.api_key = "api_key"  # Replace with your actual key


In [5]:
from google.colab import files
uploaded = files.upload()

Saving glove.jpg to glove (1).jpg


In [6]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests

# Step 1: Load BLIP Processor and Model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Step 2: Define a Function to Process an Image
def generate_caption(image):
    """Generate a caption for an input image using BLIP."""
    inputs = processor(image, return_tensors="pt")
    outputs = model.generate(**inputs)
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    return caption

# Testing with a Sample Image
image = Image.open("glove.jpg")
# image = Image.open(requests.get(image, stream=True).raw)

caption = generate_caption(image)
print("Generated Caption:", caption)


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Caption: the north face leather gloves


In [13]:
!pip install openai==0.28  #to tackle this:You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0



In [15]:
import openai

try:
    openai.api_key = api_key
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": "Test connection"}]
    )
    print("Response:", response.choices[0].message["content"])
except openai.error.AuthenticationError:
    print("Authentication failed: Invalid API Key")
except Exception as e:
    print(f"Error: {e}")


Response: Your connection seems to be working fine! How can I assist you today?


In [9]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import openai
import gradio as gr

# OpenAI API Key
openai.api_key = api_key  # Replace with your valid OpenAI API key

# Load BLIP Model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Generate Caption with BLIP
def generate_caption(image):
    """Generate a caption for an input image using BLIP."""
    try:
        inputs = processor(image, return_tensors="pt")
        outputs = model.generate(**inputs)
        caption = processor.decode(outputs[0], skip_special_tokens=True)
        return caption
    except Exception as e:
        return f"Error generating caption: {str(e)}"

# Query GPT Assistant with Custom Prompt
def query_gpt(caption):
    """Query GPT Assistant with a detailed prompt for structured output."""
    try:
        prompt = f"""
        You are tasked with determining the value of an item based on the text provided below. Use the item details to estimate its market value in an online marketplace, such as Poshmark or eBay. Do not ask the user for any additional information; only use the details provided.

        Description:
        {caption}

        In your output, provide the following details:
        Title of the item
        Category
        Color
        Brand
        Size (if clothing)
        Year manufactured (approximate value is fine if the exact year is unknown)
        Distinguishing characteristics (any unique or notable features)
        Approximate value (estimated market value)
        Average value of similar items (e.g., on marketplaces like Poshmark, eBay)
        Suggested listing price (based on the average market value and item condition)

        Additionally, notify the user if there are any versions of the item that may be worth more. For example, if the item has distinguishing characteristics like being rare, recalled, misprinted, or any other special feature, it may fetch a higher value than the typical version.

        You MUST include:
        Average sold prices from Poshmark in the last 90 days.
        Average sold prices from eBay in the last 90 days.
        Approximate Sell-Through Rate (STR) for the last 90 days.

        """
        response = openai.ChatCompletion.create(
            model="gpt-4-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.7,
        )
        return response.choices[0].message["content"]
    except Exception as e:
        return f"Error querying GPT: {str(e)}"

# Process Image with BLIP and GPT
def process_image(image):
    """Generate caption with BLIP and query GPT Assistant for detailed output."""
    caption = generate_caption(image)
    if "Error" in caption:
        return caption, "Error: Skipped GPT due to BLIP error."
    gpt_response = query_gpt(caption)
    return caption, gpt_response

# Gradio Interface
with gr.Blocks() as demo:
    with gr.Row():
        gr.Markdown("# Image to Product Details Assistant")
    with gr.Row():
        image_input = gr.Image(type="pil", label="Upload an Image")
    with gr.Row():
        caption_output = gr.Textbox(label="BLIP's Generated Caption")
        gpt_output = gr.Textbox(label="GPT Assistant Response")
    process_button = gr.Button("Process Image")
    process_button.click(process_image, inputs=image_input, outputs=[caption_output, gpt_output])

demo.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:519: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializa

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fde3f097530da7353f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
